In [3]:
!pip install faiss-cpu


[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from retrievers.DPR import DPRRetriever
from Evaluation import Evaluation

In [5]:
doc_path = "../dataset/CISI.ALL"
qry_path = "../dataset/CISI.QRY"
rel_path = "../dataset/CISI.REL"

dpr_retriever = DPRRetriever(doc_path, qry_path, rel_path)

C:\Users\marcu\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeque

In [ ]:
retrieved_docs = {}
relevant_docs = dpr_retriever.rel_set

for _, row in dpr_retriever.qry_set.iterrows():
    qid = row["query_id"]
    qtext = row["text"]
    num_relevant_docs = len(relevant_docs.get(qid, set()))
    retrieved_results = dpr_retriever._retrieve_dpr(qtext)
    print(retrieved_results)
    sliced_retrieved_results = retrieved_results[:num_relevant_docs]

    retrieved_docs[qid] = [result["doc_id"] for result in sliced_retrieved_results]

retrieved_docs = {str(qid): [str(doc_id) for doc_id in docs] for qid, docs in retrieved_docs.items()}
relevant_docs = {str(qid): {str(doc_id) for doc_id in docs} for qid, docs in relevant_docs.items()}

In [ ]:
query_ids = list(relevant_docs.keys())
evaluator = Evaluation(retrieved_docs, relevant_docs, query_ids)

results_file = '../results/bm25_evaluation.txt'
evaluator.save_evaluation_results(results_file)